In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

# Create a sample dataset (you can replace this with your actual data)
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Define a CNN model with a fully connected layer for classification
class CNNClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(CNNClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, output_size)  # Adjust the input size based on your data

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Define hyperparameters
input_size = 90 # Input size: 31 sequences of length 3
output_size = 90  # Number of classes
learning_rate = 0.0001
batch_size = 256
num_epochs = 1000

# Generate sample data (replace with your actual data)
# For this example, we'll generate random data
x = np.load('x2.npy')
print(x.shape)
# print(np.max(x))
# x = x + np.random.uniform(-0.5*np.max(x), 0.5*np.max(x))
x = np.squeeze(x).reshape(x.shape[0], -1).astype('float32') 
y = np.load('y2.npy').astype('float32')

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Create data loaders
train_dataset = CustomDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize the model and optimizer
model = CNNClassifier(input_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_list = []
# Training loop
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    loss_list.append(loss.item())
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
np.save('loss.npy', loss_list)
torch.save(model.state_dict(), 'slagent.pth')




(14049, 90)
Epoch [1/1000], Loss: 41.22433090209961
Epoch [2/1000], Loss: 14.70810317993164
Epoch [3/1000], Loss: 9.637167930603027
Epoch [4/1000], Loss: 6.873147964477539
Epoch [5/1000], Loss: 6.443809986114502
Epoch [6/1000], Loss: 4.316901206970215
Epoch [7/1000], Loss: 3.010817527770996
Epoch [8/1000], Loss: 2.4873080253601074
Epoch [9/1000], Loss: 3.1820180416107178
Epoch [10/1000], Loss: 1.7218821048736572
Epoch [11/1000], Loss: 2.211937665939331
Epoch [12/1000], Loss: 1.2673730850219727
Epoch [13/1000], Loss: 1.5399022102355957
Epoch [14/1000], Loss: 1.2414360046386719
Epoch [15/1000], Loss: 1.0599056482315063
Epoch [16/1000], Loss: 0.9435206651687622
Epoch [17/1000], Loss: 1.0878024101257324
Epoch [18/1000], Loss: 0.6988207101821899
Epoch [19/1000], Loss: 0.772471010684967
Epoch [20/1000], Loss: 0.9426027536392212
Epoch [21/1000], Loss: 0.7583367824554443
Epoch [22/1000], Loss: 0.6268842816352844
Epoch [23/1000], Loss: 0.7177212834358215
Epoch [24/1000], Loss: 0.724487900733947

In [2]:
# Test the model
test_dataset = CustomDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()
model.load_state_dict(torch.load('slagent.pth'))

correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs).numpy()
        outputs = np.rint(outputs).astype(int)
        
        # print('out: ', outputs[5])
        # print('target: ', targets.numpy()[5])
        # print(targets.shape[0])
        total += targets.shape[0]* targets.shape[1]
        for i, outvector in enumerate(outputs):
            for j, out in enumerate(outvector):
            # print(out)
                # print(targets[i, j].numpy())
                if out == int(targets[i, j].numpy()):
                    correct += 1

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy}%')

Test Accuracy: 97.61842625543693%


In [3]:
outputs.shape

(250, 90)

In [9]:
x.shape

(1310, 90)